# Use our crops and weather data to build a predictive model

In [1]:
# load all needed libraries

import pandas as pd
import pickle
#import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler, MinMaxScaler
#from sklearn.cluster import KMeans
#from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
#from sklearn.neighbors import KNeighborsClassifier
#from sklearn.svm import SVC
#from sklearn.tree import DecisionTreeClassifier
#from sklearn.ensemble import RandomForestClassifier
#from sklearn.ensemble import GradientBoostingClassifier
#from sklearn.metrics import classification_report
#from sklearn.model_selection import cross_val_score
#from sklearn.model_selection import GridSearchCV
#from sklearn.model_selection import StratifiedKFold
from sklearn.linear_model import LinearRegression
from sklearn.svm import SVR
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.metrics import mean_squared_error, r2_score,explained_variance_score,mean_absolute_error
#from sklearn.model_selection import GridSearchCV

pd.set_option('display.max_columns', None)

In [2]:
# import data

folder = 'C:/Users/szums/AIBootCampPrime/AgProject3/Resources/'

#crops data
cropsdf = pd.read_csv(folder + 'cropsANNUALNCwideV2.csv',delimiter=',') 
cropsdf['YEAR'] = pd.to_numeric(cropsdf['YEAR'],errors='coerce')
display(cropsdf.info())

print(f'{len(cropsdf)} rows read from crops file.')

#weather data
wxdf = pd.read_excel(folder + 'MergedWeatherDataAnnual.xlsx')

print(f'{len(wxdf)} rows read from weather file.')


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 160 entries, 0 to 159
Data columns (total 73 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   YEAR                    158 non-null    float64
 1   BARLEY_ACRES            98 non-null     float64
 2   BARLEY_$                75 non-null     float64
 3   BARLEY_BU               100 non-null    float64
 4   BARLEY_BU_ACRE          101 non-null    float64
 5   BARLEY_$_ACRE           75 non-null     float64
 6   BARLEY_CALC_ACRES       100 non-null    float64
 7   CORN_ACRES              98 non-null     float64
 8   CORN_$                  76 non-null     float64
 9   CORN_BU                 158 non-null    float64
 10  CORN_BU_ACRE            158 non-null    float64
 11  CORN_$_ACRE             76 non-null     float64
 12  CORN_CALC_ACRES         158 non-null    float64
 13  COTTON_ACRES            115 non-null    float64
 14  COTTON_$                14 non-null     fl

None

160 rows read from crops file.
45 rows read from weather file.


In [3]:
# we have weather data from 2000 - 2023 and we have crops data from 1866 to 2023.  make the years in each file match up.

wxdf1 = wxdf.dropna()

wxminyear = wxdf1['year'].min()
wxmaxyear = wxdf1['year'].max()
print('min year in wx file',wxminyear,'max year',wxmaxyear)

crminyear = cropsdf['YEAR'].min()
crmaxyear = cropsdf['YEAR'].max()
print('min year in crop file',crminyear,'max year',crmaxyear)

cropsdf1 = cropsdf[(cropsdf['YEAR'] >= wxminyear) & (cropsdf['YEAR'] <= wxmaxyear)]

if len(wxdf1) == len(cropsdf1):
    print('Matching rows')
else:
    print('ERROR: wxdf1 and cropsdf1 have different numbers of years')

#display('cropsdf1',cropsdf1.shape)
#display('wxdf1',wxdf1.shape)


min year in wx file 2000 max year 2020
min year in crop file 1866.0 max year 2023.0
Matching rows


In [4]:
# make sure the dataframes are sorted by year

wx_sorted = wxdf1.sort_values(by=['year'])

crops_sorted = cropsdf1.sort_values(by=['YEAR'])

# wx_sorted.drop(columns=['Weeks of Exceptional Drought', 'Weeks of Severe drought', 'Weeks of Extreme Drought'], axis=1,inplace=True)
# performs better with these drought factors left in

wx_sorted.columns

Index(['year', 'Fall (SON)', 'Spring (MAM)', 'Summer (JJA)', 'Winter (DJF)',
       'Annual Avg', 'precp Annual avg', 'Precip Fall', 'Precip Spring',
       'Precip Summer', 'Precip Winter', 'Weeks of Severe drought',
       'Weeks of Extreme Drought', 'Weeks of Exceptional Drought'],
      dtype='object')

In [5]:
# now work with target values one by one

# targets = ['BARLEY_BU_ACRE','CORN_BU_ACRE','COTTON_LB_ACRE','HAY_T_ACRE','OATS_BU_ACRE','PEANUTS_LB_ACRE','PEPPERS, BELL_CWT_ACRE',\
#             'SOYBEANS_BU_ACRE','SQUASH_CWT_ACRE','SWEET_CWT_ACRE','TOBACCO_LB_ACRE','WHEAT_BU_ACRE']

X = wx_sorted.drop(['year','Annual Avg','precp Annual avg'],axis='columns')
display(X.columns)

#columns_to_scale = ['']
crops_sorted.dropna(axis=1, how='any',inplace=True)

#targets = crops_sorted.columns.drop('YEAR')
targets = ['BARLEY_$_ACRE','CORN_$_ACRE','HAY_$_ACRE','OATS_$_ACRE','PEANUTS_$_ACRE','PEPPERS,BELL_$_ACRE',
          'SOYBEANS_$_ACRE','SQUASH_$_ACRE','SWEET_$_ACRE','TOBACCO_$_ACRE','WHEAT_$_ACRE','BARLEY_BU_ACRE',
          'CORN_BU_ACRE','COTTON_LB_ACRE','HAY_T_ACRE','OATS_BU_ACRE','PEANUTS_LB_ACRE','PEPPERS, BELL_CWT_ACRE',
          'SOYBEANS_BU_ACRE','SQUASH_CWT_ACRE','SWEET_CWT_ACRE','TOBACCO_LB_ACRE','WHEAT_BU_ACRE']
# removed 'COTTON_$_ACRE' because of missing data.

display(targets)

Index(['Fall (SON)', 'Spring (MAM)', 'Summer (JJA)', 'Winter (DJF)',
       'Precip Fall', 'Precip Spring', 'Precip Summer', 'Precip Winter',
       'Weeks of Severe drought', 'Weeks of Extreme Drought',
       'Weeks of Exceptional Drought'],
      dtype='object')

['BARLEY_$_ACRE',
 'CORN_$_ACRE',
 'HAY_$_ACRE',
 'OATS_$_ACRE',
 'PEANUTS_$_ACRE',
 'PEPPERS,BELL_$_ACRE',
 'SOYBEANS_$_ACRE',
 'SQUASH_$_ACRE',
 'SWEET_$_ACRE',
 'TOBACCO_$_ACRE',
 'WHEAT_$_ACRE',
 'BARLEY_BU_ACRE',
 'CORN_BU_ACRE',
 'COTTON_LB_ACRE',
 'HAY_T_ACRE',
 'OATS_BU_ACRE',
 'PEANUTS_LB_ACRE',
 'PEPPERS, BELL_CWT_ACRE',
 'SOYBEANS_BU_ACRE',
 'SQUASH_CWT_ACRE',
 'SWEET_CWT_ACRE',
 'TOBACCO_LB_ACRE',
 'WHEAT_BU_ACRE']

In [6]:
# peek into the X and y structures to get some data for external testing

# display(X_test.head())
# display(y_test.head())

# X_test.to_csv(folder + 'sampleX.csv',index=False)
# y_test.to_csv(folder + 'sampley.csv',index=False)

In [7]:

mse_list = []
r2_list = []
model_list = []
target_list = []
ev_list = []
mae_list = []
score_list = []

models = [LinearRegression,SVR,DecisionTreeRegressor,RandomForestRegressor,GradientBoostingRegressor]
model_names = ['LinearRegression()','SVR()','DecisionTreeRegressor()','RandomForestRegressor()','GradientBoostingRegressor()']
model_file_names = ['lr','svr','dtr','rfr','gbr']

for t in targets:
    print('Working on',t)
    y = crops_sorted[t]
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=98)

    scaler_X = StandardScaler()
    X_train_scaled = scaler_X.fit_transform(X_train)
    X_test_scaled = scaler_X.fit_transform(X_test)
    #X_train_scaled = X_train
    #X_test_scaled = X_test

    scaler_y = StandardScaler()
    y_train_scaled = scaler_y.fit_transform(y_train.values.reshape(-1, 1)).flatten()
    y_test_scaled = scaler_y.fit_transform(y_test.values.reshape(-1, 1)).flatten()
    #y_train_scaled = y_train
    #y_test_scaled = y_test

    for m in range(len(models)):
        
        target_list.append(t)

        this_model = models[m]()
        this_model_name = model_names[m]
        this_model_file_name = model_file_names[m]
        #print('...',this_model_name,this_model)

        this_model.fit(X_train_scaled,y_train_scaled)
        y_pred = this_model.predict(X_test_scaled)  # HERE IS THE MODEL

        this_score = this_model.score(X_train_scaled,y_train_scaled)
        score_list.append(this_score)
        #display(y_pred.shape)

        y_pred_original = scaler_y.inverse_transform(y_pred.reshape(-1, 1))
        #display(y_pred_original.shape)

        model_list.append(this_model_name)

        mse = mean_squared_error(y_test_scaled,y_pred)
        mse_list.append(mse)

        r2 = r2_score(y_test_scaled, y_pred)
        r2_list.append(r2)

        mae = mean_absolute_error(y_test_scaled,y_pred)
        mae_list.append(mae)

        ev_score = explained_variance_score(y_test_scaled,y_pred)
        ev_list.append(ev_score)

        # prepare to save the trained model
        model_save_name = folder + t + this_model_file_name + "_model.pkl"
        scalar_X_save_name = folder + t + this_model_file_name + "_X.pkl"
        scalar_y_save_name = folder + t + this_model_file_name + "_y.pkl"

        with open(model_save_name, 'wb') as file:
            pickle.dump(this_model, file)
        with open(scalar_X_save_name, 'wb') as file:
            pickle.dump(scaler_X, file)
        with open(scalar_y_save_name, 'wb') as file:
            pickle.dump(scaler_y, file)

        print('saved model and scalars',model_save_name)

    

resultsdf = pd.DataFrame({'Target':target_list,
                          "model" : model_list,
                          'MSE':mse_list,
                          'R2': r2_list,
                          'Mean_abs_error': mae_list,
                          'Explained var' : ev_list,
                          'score': score_list})
    


Working on BARLEY_$_ACRE
saved model and scalars C:/Users/szums/AIBootCampPrime/AgProject3/Resources/BARLEY_$_ACRElr_model.pkl
saved model and scalars C:/Users/szums/AIBootCampPrime/AgProject3/Resources/BARLEY_$_ACREsvr_model.pkl
saved model and scalars C:/Users/szums/AIBootCampPrime/AgProject3/Resources/BARLEY_$_ACREdtr_model.pkl
saved model and scalars C:/Users/szums/AIBootCampPrime/AgProject3/Resources/BARLEY_$_ACRErfr_model.pkl
saved model and scalars C:/Users/szums/AIBootCampPrime/AgProject3/Resources/BARLEY_$_ACREgbr_model.pkl
Working on CORN_$_ACRE
saved model and scalars C:/Users/szums/AIBootCampPrime/AgProject3/Resources/CORN_$_ACRElr_model.pkl
saved model and scalars C:/Users/szums/AIBootCampPrime/AgProject3/Resources/CORN_$_ACREsvr_model.pkl
saved model and scalars C:/Users/szums/AIBootCampPrime/AgProject3/Resources/CORN_$_ACREdtr_model.pkl
saved model and scalars C:/Users/szums/AIBootCampPrime/AgProject3/Resources/CORN_$_ACRErfr_model.pkl
saved model and scalars C:/Users/sz

In [8]:
resultsdf.to_excel(folder+'cropWeatherModelResults5.xlsx', index=False)
print('finished')


finished


In [9]:
# This code tries to determine best model to use by looking at average quality metrics for instances where
# the model is used.  However, since the model performs very differently for each target, this approach is abandoned.
#display(resultsdf.head())

meanslist = []
qmeasurelist = []

for t in ['MSE','R2','Mean_abs_error','Explained var']:
    print('\nComparing model accuracy. Measure:',t)
    modelmeans = resultsdf.groupby('model')[t].mean()
    if t in ['MSE','Mean_abs_error']:
        modelmeans.sort_values(ascending=True,inplace=True)
        print(f'for {t}, the best values are as low a possible.')
    else:
        modelmeans.sort_values(ascending=False,inplace=True)
        print(f'for {t}, the best values are as close to 1.0 as possible.')
    #meanslist.append(modelmeans)
    #qmeasurelist.append(t)
    print(modelmeans)

#resultssummary = pd.DataFrame({"Quality Measure" : qmeasurelist,
#                               "MeanModelAccuracy" : meanslist})
display(modelmeans)
#resultssummary


Comparing model accuracy. Measure: MSE
for MSE, the best values are as low a possible.
model
SVR()                          0.918014
RandomForestRegressor()        1.094092
GradientBoostingRegressor()    1.355727
DecisionTreeRegressor()        1.719804
LinearRegression()             3.011258
Name: MSE, dtype: float64

Comparing model accuracy. Measure: R2
for R2, the best values are as close to 1.0 as possible.
model
SVR()                          0.081986
RandomForestRegressor()       -0.094092
GradientBoostingRegressor()   -0.355727
DecisionTreeRegressor()       -0.719804
LinearRegression()            -2.011258
Name: R2, dtype: float64

Comparing model accuracy. Measure: Mean_abs_error
for Mean_abs_error, the best values are as low a possible.
model
SVR()                          0.808911
RandomForestRegressor()        0.855637
GradientBoostingRegressor()    0.938516
DecisionTreeRegressor()        1.015326
LinearRegression()             1.369951
Name: Mean_abs_error, dtype: float64


model
SVR()                          0.101164
RandomForestRegressor()       -0.074893
GradientBoostingRegressor()   -0.294190
DecisionTreeRegressor()       -0.592516
LinearRegression()            -2.011258
Name: Explained var, dtype: float64

# Model performance summary

Model performance was measured by Mean Square Error, R2, Mean Absolute Error and Explained Variance

When used to model __crop value per acre__, the performance of each model varied significantly from crop to crop.  With these target measures, the crops that were most effectivly modeled are: Soybeans, Tobacco, Peanuts and Squash.  (Cotton could not be modeled due to absence of recent $ data.) The models that performed best with these crops were DecisionTreeRegressor and GradientBoostingRegressor.  

When used to model crop __production per acre__ (bushels, tons, cwt, etc) per acre, the models also varied from crop to crop. With these measures, the models did the best job with Soybeans, Hay, Tobacco and Cotton.  The most effective models were GradientBoosting, SVR and RandomForestRegressor.

This is a list of target and best performing model (R2):

| target	| best model |
|-------------------|-----------------------------|
| SOYBEANS_$_ACRE	| GradientBoostingRegressor() | 
| TOBACCO_$_ACRE	| GradientBoostingRegressor() | 
| SQUASH_$_ACRE	| LinearRegression() | 
| PEANUTS_$_ACRE	| DecisionTreeRegressor() | 
| BARLEY_$_ACRE	| RandomForestRegressor() | 
| HAY_$_ACRE	| LinearRegression() | 
| PEPPERS,BELL_$_ACRE	| SVR() | 
| SWEET_$_ACRE	| GradientBoostingRegressor() | 
| OATS_$_ACRE	| RandomForestRegressor() | 
| CORN_$_ACRE	| SVR() | 
| WHEAT_$_ACRE	| RandomForestRegressor() | 

Note that these may change with further testing.